# Face Detection Analysis

On this notebook the important elements to analyse are:
* **Detection time reltaive to frame rate and area:** The frame size and the total duration of the video impact on the time spent detecting. We can change the resolution but not the length of the video, instead we can change the frequency the detection happens.
* **Face area relative to frame scale and frame size threshold:** Changing the resolution of the video can impact on the number of faces we detect.
* **Content delta distribution to select threshold:** For the next stage of the pipeline a threshold is required for shot transitions.

In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt